In [1]:
import os
os.environ["OMP_NUM_THREADS"] = "4"
# # os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
# # os.environ["CUDA_VISIBLE_DEVICES"] = "1"
os.chdir('app/')

In [2]:
# import os
# os.environ["OMP_NUM_THREADS"] = "4"
# os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
# os.environ["CUDA_VISIBLE_DEVICES"] = "1"

import pandas as pd
import numpy as np
import torch
from functools import partial
import pytorch_lightning as pl
import warnings
warnings.filterwarnings("ignore")

from torch.utils.data import DataLoader

from ptls.data_load.datasets import MemoryMapDataset
from ptls.data_load.iterable_processing.iterable_seq_len_limit import ISeqLenLimit
from ptls.data_load.iterable_processing.to_torch_tensor import ToTorch
from ptls.data_load.iterable_processing.feature_filter import FeatureFilter
from ptls.nn import TrxEncoder, RnnSeqEncoder
from ptls.frames.coles import CoLESModule
from ptls.data_load.iterable_processing import SeqLenFilter
from ptls.frames.coles import ColesIterableDataset
from ptls.frames.coles.split_strategy import SampleSlices
from ptls.frames import PtlsDataModule
from ptls.preprocessing import PandasDataPreprocessor
from ptls.data_load.utils import collate_feature_dict
from ptls.data_load.iterable_processing_dataset import IterableProcessingDataset

from tqdm.auto import tqdm
import lightgbm as ltb

from src.preprocessing.pandas_preprocessor import CustomPandasDataPreprocessor

In [3]:
from datetime import datetime

In [4]:
transactions_train = pd.read_parquet("data_preprocessed/trx_train_preprocessed.parquet")
# transactions_test = pd.read_parquet("data_preprocessed/trx_test_preprocessed.parquet")

geo_train = pd.read_parquet("data_preprocessed/geo_train_preprocessed.parquet")
dial_train = pd.read_parquet("data/dial_train.parquet")

In [7]:
dial_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1215209 entries, 0 to 1215208
Data columns (total 3 columns):
 #   Column      Non-Null Count    Dtype         
---  ------      --------------    -----         
 0   client_id   1215209 non-null  object        
 1   event_time  1215209 non-null  datetime64[us]
 2   embedding   1215209 non-null  object        
dtypes: datetime64[us](1), object(2)
memory usage: 27.8+ MB


In [8]:
dial_train_fix = pd.concat(
    [
        dial_train.drop(columns=['embedding']), 
        pd.DataFrame(np.vstack(dial_train.embedding.to_list()), columns=[f'emb_{i}' for i in range(768)])
    ],
    axis=1
) 

In [9]:
geo_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 528278383 entries, 0 to 528278382
Data columns (total 10 columns):
 #   Column       Dtype         
---  ------       -----         
 0   client_id    object        
 1   event_time   datetime64[ns]
 2   time_diff    int16         
 3   month        int8          
 4   day_of_week  int8          
 5   hour         int8          
 6   minute       int8          
 7   geohash_4    int16         
 8   geohash_5    int32         
 9   geohash_6    int32         
dtypes: datetime64[ns](1), int16(2), int32(2), int8(4), object(1)
memory usage: 15.7+ GB


In [10]:
transactions_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 167467698 entries, 0 to 167467697
Data columns (total 19 columns):
 #   Column         Dtype         
---  ------         -----         
 0   event_time     datetime64[ns]
 1   amount         float32       
 2   client_id      object        
 3   event_type     int8          
 4   event_subtype  int8          
 5   currency       int8          
 6   src_type11     int16         
 7   src_type12     int16         
 8   dst_type11     int16         
 9   dst_type12     int16         
 10  src_type21     int32         
 11  src_type22     int8          
 12  src_type31     int16         
 13  src_type32     int8          
 14  time_diff      int16         
 15  month          int8          
 16  day_of_week    int8          
 17  hour           int8          
 18  minute         int8          
dtypes: datetime64[ns](1), float32(1), int16(6), int32(1), int8(9), object(1)
memory usage: 7.0+ GB


In [11]:
target = pd.read_parquet('data/train_target.parquet')
target['mon'] = pd.to_datetime(target['mon'])

In [12]:
target.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10246704 entries, 0 to 10246703
Data columns (total 6 columns):
 #   Column     Dtype         
---  ------     -----         
 0   mon        datetime64[ns]
 1   target_1   int32         
 2   target_2   int32         
 3   target_3   int32         
 4   target_4   int32         
 5   client_id  object        
dtypes: datetime64[ns](1), int32(4), object(1)
memory usage: 312.7+ MB


In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer


def generate_tfidf_features(df, col, max_features):
    bot = df.groupby('client_id')[col].apply(lambda x: list(x))
    corpus = [' '.join(map(str, x)) for x in bot]

    tfidf = TfidfVectorizer(max_features=max_features)
    tfidf.fit(corpus)

    tfidf_features = pd.DataFrame(
        tfidf.transform(corpus).A,
        columns=[f'tfidf_{col}_{k}' for k in tfidf.get_feature_names_out()]
    )
    tfidf_features['client_id'] = bot.index

    return tfidf_features


def generate_pivot_features(df, col, value, aggfunc):
    pivot = df.pivot_table(values=[value], index=['client_id'], columns=[col], aggfunc=aggfunc)
    pivot.columns = [f'{col}_{v}_{value}_{aggfunc}' for _, v in pivot.columns]
    pivot = pivot.reset_index()

    return pivot


def geohash_counter(df, max):
    if df.empty:
        # Create an empty DataFrame with the correct structure
        columns = ['client_id'] + [f'geohash_4_{i}' for i in range(1, max + 1)]
        return pd.DataFrame(columns=columns)
    
    def counter(x, max):
        counts, _ = np.histogram(x, bins=np.arange(1, max + 2))
        return counts

    counts = df.groupby('client_id')['geohash_4'].apply(counter, max=max)

    counts = pd.DataFrame(
        np.vstack(counts.to_list()), 
        columns=[f'geohash_4_{i}' for i in range(1, max + 1)],
        index=counts.index
    )

    return counts.reset_index()

In [14]:
dial_train_fix['event_time'].max()

Timestamp('2022-12-30 20:50:45.829493')

In [15]:
transactions_train['event_time'].max()

Timestamp('2022-12-20 22:59:53.424424')

In [16]:
start_date = '2022-01-01'
end_date = '2023-01-31'

# Generate a date range with the end of each month
dates = pd.date_range(start=start_date, end=end_date, freq='M')

# Convert the dates to numpy datetime64 array
end_of_months = dates.to_numpy(dtype='datetime64[ns]')

print(end_of_months)

['2022-01-31T00:00:00.000000000' '2022-02-28T00:00:00.000000000'
 '2022-03-31T00:00:00.000000000' '2022-04-30T00:00:00.000000000'
 '2022-05-31T00:00:00.000000000' '2022-06-30T00:00:00.000000000'
 '2022-07-31T00:00:00.000000000' '2022-08-31T00:00:00.000000000'
 '2022-09-30T00:00:00.000000000' '2022-10-31T00:00:00.000000000'
 '2022-11-30T00:00:00.000000000' '2022-12-31T00:00:00.000000000'
 '2023-01-31T00:00:00.000000000']


In [18]:
dataframes = []

# dates_sorted = np.sort(target['mon'].unique())

for i in range(1, len(end_of_months)):
    trx_before_date = transactions_train[
        (transactions_train['event_time'] <= end_of_months[i]) & 
        (transactions_train['event_time'] > end_of_months[i-1])
    ]

    geo_before_date = geo_train[
        (geo_train['event_time'] <= end_of_months[i]) & 
        (geo_train['event_time'] > end_of_months[i-1])
    ]

    dial_before_data = dial_train_fix[
        (dial_train['event_time'] <= end_of_months[i]) & 
        (dial_train['event_time'] > end_of_months[i-1])
    ]


    # amount_features = before_date.groupby('client_id')['amount'].agg(['mean', 'std']).reset_index()
    event_type_amount_mean = generate_pivot_features(trx_before_date, 'event_type', 'amount', 'mean')
    # event_type_amount_std = generate_pivot_features(trx_before_date, 'event_type', 'amount', 'std')
    # event_type_tfidf = generate_tfidf_features(before_date, 'event_type', 100)

    geo_counts = geohash_counter(geo_before_date, 100)

    dial_mean = dial_before_data.drop(columns=['event_time']).groupby('client_id').mean().reset_index().iloc

    df = (
        target[target['mon'] == end_of_months[i]]
        .merge(event_type_amount_mean, on='client_id', how='left')
        .merge(geo_counts, on='client_id', how='left')
        .merge(dial_mean, on='client_id', how='left')
    )

    dataframes.append(df)

In [18]:
df = pd.concat(dataframes, axis=0)

NameError: name 'dataframes' is not defined

In [20]:
from sklearn.model_selection import train_test_split

X = df.drop(columns=['mon', 'target_1', 'target_2', 'target_3', 'target_4', 'client_id']).fillna(0)
y = df[['target_1', 'target_2', 'target_3', 'target_4']]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=69)

In [21]:
from lightgbm import LGBMClassifier
from sklearn.linear_model import LogisticRegression

from sklearn.metrics import roc_auc_score

params = {
    "n_estimators": 500,
      "boosting_type": "gbdt",
      "objective": "binary",
      "subsample": 0.5,
      "subsample_freq": 1,
      "learning_rate": 0.02,
      "feature_fraction": 0.75,
      "max_depth": 6,
      "lambda_l1": 1,
      "lambda_l2": 1,
      "min_data_in_leaf": 50,
      "random_state": 42,
      "n_jobs": 8,
      "verbose": 0,
}


for i in range(1, 5):
    model = LGBMClassifier(verbose=0)
    model.fit(X_train, y_train[f'target_{i}'])

    print(f'Target {i}')
    print("Train ROC-AUC:", roc_auc_score(y_train[f'target_{i}'], model.predict_proba(X_train)[:, 1]))
    print("Test ROC-AUC:", roc_auc_score(y_test[f'target_{i}'], model.predict_proba(X_test)[:, 1]), end='\n\n')

: 

: 

In [62]:
pd.Series(model.feature_importances_, index=X_train.columns).sort_values().to_csv('imp.csv')

In [33]:
np.sort(target['mon'].unique())[0]

0             True
1             True
2             True
3             True
4             True
             ...  
167467693    False
167467694    False
167467695    False
167467696    False
167467697    False
Name: event_time, Length: 167467698, dtype: bool

In [6]:
%%time

preprocessor = CustomPandasDataPreprocessor(
    col_id="client_id",
    col_event_time="event_time",
    event_time_transformation="dt_to_timestamp",
    cols_category=["event_type",
                   "event_subtype",
                   "currency",
                   "src_type11",
                   "src_type12",
                   "dst_type11",
                   "dst_type12",
                   "src_type21",
                   "src_type22",
                   "src_type31",
                   "src_type32"],
    cols_identity=["amount", "time_diff", "month", "day_of_week", "hour", "minute"],
    return_records=False,
)

processed_train = preprocessor.fit_transform(transactions_train)

processed_test = preprocessor.transform(transactions_test)

  0%|          | 0/855627 [00:00<?, ?it/s]

  0%|          | 0/225744 [00:00<?, ?it/s]

CPU times: user 23min 18s, sys: 3min 11s, total: 26min 29s
Wall time: 26min 25s


In [ ]:
target_train = pd.read_parquet("data/train_target.parquet")

target_preprocessor = PandasDataPreprocessor(
    col_id="client_id",
    col_event_time="mon",
    event_time_transformation="dt_to_timestamp",
    cols_identity=["target_1", "target_2", "target_3", "target_4"],
    return_records=False,
)

processed_target = target_preprocessor.fit_transform(target_train)

In [ ]:
test_target_b = pd.read_parquet("data/test_target_b.parquet")

**Обработка датасета:**

- Транзакции, у которых размер < min_seq_len выкидываются
- Транзакции, у которых длина > max_seq_len, обрезаются и конвертируются в torch.tensor
- Не нужные для CoLES фичи удаляются

In [9]:
train = MemoryMapDataset(
    data=processed_train.to_dict("records"),
    i_filters=[
        FeatureFilter(drop_feature_names=['client_id', 'target_1', 'target_2', 'target_3', 'target_4']),
        SeqLenFilter(min_seq_len=32),
        ISeqLenLimit(max_seq_len=4096),
        ToTorch()
    ]
)

test = MemoryMapDataset(
    data=processed_test.to_dict("records"),
    i_filters=[
        FeatureFilter(drop_feature_names=['client_id', 'target_1', 'target_2', 'target_3', 'target_4']),
        SeqLenFilter(min_seq_len=32),
        ISeqLenLimit(max_seq_len=4096),
        ToTorch()
    ]
)

In [10]:
train_ds = ColesIterableDataset(
    data=train,
    splitter=SampleSlices(
        split_count=5,
        cnt_min=32,
        cnt_max=180
    )
)

valid_ds = ColesIterableDataset(
    data=test,
    splitter=SampleSlices(
        split_count=5,
        cnt_min=32,
        cnt_max=180
    )
)

In [11]:
train_dl = PtlsDataModule(
    train_data=train_ds,
    train_num_workers=16,
    train_batch_size=256,
    valid_data=valid_ds,
    valid_num_workers=16,
    valid_batch_size=256
)

# Model

- numeric_values обрабатываются как BatchNorm+Linear
- embedidngs - nn.Embedidngs

In [ ]:
trx_encoder_params = dict(
    embeddings_noise=0.003,
    numeric_values={
        'amount': 'identity',
        'time_diff': 'identity',
        'month': 'identity',
        'day_of_week': 'identity', 
        'hour': 'identity', 
        'minute': 'identity',
    },
    embeddings={
        "event_type": {'in': 50, "out": 24},
        "event_subtype": {'in': 50, "out": 24},
        'currency': {'in': 10, 'out': 24},
        'src_type11': {'in': 30, 'out': 24},
        'src_type12': {'in': 100, 'out': 24},
        'dst_type11': {'in': 30, 'out': 24},
        'dst_type12': {'in': 100, 'out': 24},
        'src_type22': {'in': 100, 'out': 24},
        # 'src_type21': {'in': 30000, 'out': 24},
        'src_type31': {'in': 1500, 'out': 24},
        'src_type32': {'in': 100, 'out': 24},
    }
)

- **TrxEncoder** - обрабатывает каждую тразнакцию (строит для неё эмбеддиг)
- **SeqEncoder** - обрабатывает последовательность

In [ ]:
seq_encoder = RnnSeqEncoder(
    trx_encoder=TrxEncoder(**trx_encoder_params),
    hidden_size=256,
    type='gru',
)

In [ ]:
from src.modules.masked_coles_module import MaskedCoLESModule

# model = MaskedCoLESModule(
#     lam=0.05,
#     seq_encoder=seq_encoder,
#     optimizer_partial=partial(torch.optim.Adam, lr=0.001),
#     lr_scheduler_partial=partial(torch.optim.lr_scheduler.StepLR, step_size=3, gamma=0.9025)
# )

model = CoLESModule(
    seq_encoder=seq_encoder,
    optimizer_partial=partial(torch.optim.Adam, lr=0.001),
    lr_scheduler_partial=partial(torch.optim.lr_scheduler.StepLR, step_size=3, gamma=0.9025)
)

In [ ]:
# model.load_state_dict(torch.load('coles_cmlm_time.pt'))

model.load_state_dict(torch.load('coles_time.pt'))

<All keys matched successfully>

# Train

In [20]:
trainer = pl.Trainer(
    max_epochs=30,
    limit_val_batches=5000,
#     gpus=[0],
    devices='auto',
    enable_progress_bar=True,
    gradient_clip_val=0.5,
    logger=pl.loggers.TensorBoardLogger(
        save_dir='./logdir',
        name='baseline_result'
    ),
    callbacks=[
        pl.callbacks.LearningRateMonitor(logging_interval='step'),
        pl.callbacks.ModelCheckpoint(every_n_train_steps=5000, save_top_k=-1),
    ]
)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [21]:
trainer.fit(model, train_dl)

You are using a CUDA device ('NVIDIA A100-SXM4-80GB') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
Missing logger folder: ./logdir/baseline_result
2024-06-10 15:34:03.810399: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-10 15:34:03.810459: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-10 15:34:03.814181: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plu

Epoch 0: 100%|██████████| 2312/2312 [02:58<00:00, 12.96it/s, v_num=0, seq_len=83.30]
Validation: |          | 0/? [00:00<?, ?it/s]
Validation DataLoader 0:  34%|███▍      | 208/605 [00:09<00:17, 22.94it/s]


Validation DataLoader 0:  69%|██████▉   | 418/605 [00:18<00:08, 23.10it/s]


Epoch 1: 100%|██████████| 2312/2312 [04:27<00:00,  8.65it/s, v_num=0, seq_len=85.50, valid/recall_top_k=0.997]
Validation: |          | 0/? [00:00<?, ?it/s]
Validation DataLoader 0:   3%|▎         | 19/605 [00:00<00:28, 20.83it/s]


Validation DataLoader 0:  38%|███▊      | 230/605 [00:10<00:17, 21.47it/s]


...

Validation DataLoader 0:  35%|███▌      | 214/605 [00:11<00:21, 18.44it/s]


Validation DataLoader 0:  70%|███████   | 424/605 [00:21<00:09, 19.34it/s]


Epoch 29: 100%|██████████| 2312/2312 [04:13<00:00,  9.12it/s, v_num=0, seq_len=85.70, valid/recall_top_k=0.997]

`Trainer.fit` stopped: `max_epochs=30` reached.


Epoch 29: 100%|██████████| 2312/2312 [04:13<00:00,  9.12it/s, v_num=0, seq_len=85.70, valid/recall_top_k=0.997]


In [22]:
torch.save(model.state_dict(), './model.pt')

# Inference

Для каждого пользователя известно 12 таргетов, инференс происходит следующим образом:

Чтобы не происходило лика нужно для каждого клиента делать срез до текущего месяца:

Берутся все тразнакции за первый месяц, им соответствует 1-ый таргет из 12,
потом берутся транзакции за первый и второй месяц пользователя и им соотвествует 2-ой таргет и так далее.
То есть для данного пользователя, имеющего транзакции за год, мы можем получить 12 эмбеддингов, каждому из которых соответствует 1 таргет

In [13]:
class GetSplit(IterableProcessingDataset):
    def __init__(
        self,
        start_month,
        end_month,
        year=2022,
        col_id='client_id',
        col_time='event_time'
    ):
        super().__init__()
        self.start_month = start_month
        self.end_month = end_month
        self._year = year
        self._col_id = col_id
        self._col_time = col_time

    def __iter__(self):
        for rec in self._src:
            for month in range(self.start_month, self.end_month+1):
                features = rec[0] if type(rec) is tuple else rec
                features = features.copy()

                if month == 12:
                    month_event_time = datetime(self._year + 1, 1, 1).timestamp()
                else:
                    month_event_time = datetime(self._year, month + 1, 1).timestamp()

                year_event_time = datetime(self._year, 1, 1).timestamp()

                mask = features[self._col_time] < month_event_time

                for key, tensor in features.items():
                    if key.startswith('target'):
                        features[key] = tensor[month - 1].tolist()
                    elif key != self._col_id:
                        features[key] = tensor[mask]

                features[self._col_id] += '_month=' + str(month)

                yield features

def collate_feature_dict_with_target(batch, col_id='client_id', targets=False):
    batch_ids = []
    target_cols = []
    for sample in batch:

        batch_ids.append(sample[col_id])
        del sample[col_id]

        if targets:
            target_cols.append([sample[f'target_{i}'] for i in range(1, 5)])
            del sample['target_1']
            del sample['target_2']
            del sample['target_3']
            del sample['target_4']

    padded_batch = collate_feature_dict(batch)
    if targets:
        return padded_batch, batch_ids, target_cols
    return padded_batch, batch_ids


class InferenceModuleMultimodal(pl.LightningModule):
    def __init__(self, model, pandas_output=True, drop_seq_features=True, model_out_name='out'):
        super().__init__()

        self.model = model
        self.pandas_output = pandas_output
        self.drop_seq_features = drop_seq_features
        self.model_out_name = model_out_name

    def forward(self, x):
        x_len = len(x)
        if x_len == 3:
            x, batch_ids, target_cols = x
        else:
            x, batch_ids = x

        out = self.model(x)
        if x_len == 3:
            target_cols = torch.tensor(target_cols)
            x_out = {
                'client_id': batch_ids,
                'target_1': target_cols[:, 0],
                'target_2': target_cols[:, 1],
                'target_3': target_cols[:, 2],
                'target_4': target_cols[:, 3],
                self.model_out_name: out
            }
        else:
            x_out = {
                'client_id': batch_ids,
                self.model_out_name: out
            }
        torch.cuda.empty_cache()

        if self.pandas_output:
            return self.to_pandas(x_out)
        return x_out

    @staticmethod
    def to_pandas(x):
        expand_cols = []
        scalar_features = {}

        for k, v in x.items():
            if type(v) is torch.Tensor:
                v = v.cpu().numpy()

            if type(v) is list or len(v.shape) == 1:
                scalar_features[k] = v
            elif len(v.shape) == 2:
                expand_cols.append(k)
            else:
                scalar_features[k] = None

        dataframes = [pd.DataFrame(scalar_features)]
        for col in expand_cols:
            v = x[col].cpu().numpy()
            dataframes.append(pd.DataFrame(v, columns=[f'{col}_{i:04d}' for i in range(v.shape[1])]))

        return pd.concat(dataframes, axis=1)

In [14]:
%%time

train = MemoryMapDataset(
    data=processed_train.merge(processed_target.drop("event_time", axis=1), on="client_id", how="inner").to_dict("records"),
    i_filters=[
        ISeqLenLimit(max_seq_len=4096),
        FeatureFilter(keep_feature_names=['client_id', 'target_1', 'target_2', 'target_3', 'target_4']),
        GetSplit(start_month=1, end_month=12),
        ToTorch(),
    ]
)

test = MemoryMapDataset(
    data=processed_test.to_dict("records"),
    i_filters=[
        ISeqLenLimit(max_seq_len=4096),
        FeatureFilter(keep_feature_names=['client_id', 'target_1', 'target_2', 'target_3', 'target_4']),
        ToTorch(),
    ]
)

: 

: 

In [ ]:
inference_train_dl = DataLoader(
        dataset=train,
        collate_fn=partial(collate_feature_dict_with_target, targets=True),
        shuffle=False,
        num_workers=0,
        batch_size=256,
    )

inference_test_dl = DataLoader(
        dataset=test,
        collate_fn=collate_feature_dict_with_target,
        shuffle=False,
        num_workers=0,
        batch_size=256,
    )

In [ ]:
model.seq_encoder.is_reduce_sequence = True

In [ ]:
inf_module = InferenceModuleMultimodal(
        model=model,
        pandas_output=True,
        drop_seq_features=True,
        model_out_name='emb',
    )

In [ ]:
trainer = pl.Trainer(max_epochs=-1)#, gpus=[1])

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [33]:
inf_test_embeddings = pd.concat(
        trainer.predict(inf_module, inference_test_dl)
    )
inf_test_embeddings.to_parquet("test.parquet", index=False, engine="pyarrow", compression="snappy")

Missing logger folder: /app/lightning_logs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]


Predicting: 0it [00:00, ?it/s]

{'amount': tensor([ 4.7717, 11.8718, 11.9163, 12.4851, 10.3940, 12.3038, 13.4196, 10.2477,
        12.0198, 12.7011, 12.7905,  9.5507, 11.4317,  8.7042, 13.7166, 13.6137,
        13.8361, 13.5999, 13.3853, 12.2811, 13.9089, 10.4469,  8.9823, 11.3969,
        13.2385, 10.5182, 12.3229, 12.5155, 13.7520,  9.3323, 13.6977,  9.1817,
        10.3253, 12.6637, 13.5179, 10.5103, 12.4929, 12.4805,  9.2340, 11.4597,
        12.8069, 11.3322, 11.6127, 11.1062, 11.3672, 10.1697, 11.1976, 11.8042,
        10.6472,  9.0874, 12.6746, 10.9119, 12.9672, 11.3203,  9.6922,  8.8675,
        12.0143, 11.1865, 11.3158, 10.8711, 10.0164, 12.0778, 10.0149, 10.8782,
        10.6511, 13.4237, 11.1257, 10.3431, 10.6586, 12.3233,  9.7224,  9.1440,
        11.8476, 12.4282, 11.4732, 12.7085, 11.5564, 11.9524,  8.5651, 10.0666,
        11.6949, 12.5095, 11.2540, 10.8246, 11.9629, 11.1280, 10.4321, 11.9390,
         9.1417, 10.8909, 11.6947, 11.3629,  9.8933, 11.3039, 12.1011,  5.4614,
        10.8932, 11.7006,  8.

KeyError: 'client_id'

In [41]:
del inf_test_embeddings

In [42]:
inf_train_embeddings = pd.concat(
        trainer.predict(inf_module, inference_train_dl)
    )

inf_train_embeddings.to_parquet("train.parquet", index=False, engine="pyarrow", compression="snappy")

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting DataLoader 0: 100%|██████████| 29121/29121 [49:56<00:00,  9.72it/s]


In [43]:
del inf_train_embeddings

Файл **sample_submission** составляется из **client_id** файла **test_target_b**. Так как не у всех пользователей может быть транзакционная история, мы для простоты заполняем их фичи нулями.

In [45]:
inf_test_embeddings = pd.read_parquet('test.parquet')

In [46]:
not_only_trx = pd.DataFrame({"client_id": test_target_b["client_id"].unique()}).merge(inf_test_embeddings, how="left").fillna(0)
not_only_trx

,client_id,emb_0000,emb_0001,emb_0002,emb_0003,emb_0004,emb_0005,emb_0006,emb_0007,emb_0008,...,emb_0246,emb_0247,emb_0248,emb_0249,emb_0250,emb_0251,emb_0252,emb_0253,emb_0254,emb_0255
0,2b7ff0c1c99cefe259ed83c5dfa0a403f2cbc88032b671...,0.032045,0.436370,1.000000,-1.0,1.0,0.060832,-1.000000,-0.753773,1.000000,...,0.999976,1.0,-0.005480,-0.138812,-0.019887,-1.0,0.982713,1.000000,-0.030167,1.0
1,0433d23e224b7a520656da6181efadb8d556bb293158c9...,0.032044,-0.992578,1.000000,-1.0,1.0,0.060832,1.000000,-0.862121,-0.996295,...,-0.791435,1.0,-0.005481,-0.138812,-0.019887,-1.0,-0.999918,-0.552822,-0.030141,1.0
2,f2ce8b292e5f9f778f3e20db7608ac76dc8812113a2631...,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0
3,4f807e8b163c653bcaeff9f925983568f4c3e6b1a1f231...,0.048867,-0.974925,1.000000,-1.0,1.0,0.060834,-0.253653,-0.047741,1.000000,...,0.779262,1.0,-0.005480,-0.138817,-0.019829,-1.0,0.997734,1.000000,-0.030151,1.0
4,64369f6f8ae1b719332ee1bfb2b454e642b2053d2c9b8a...,0.032045,0.999924,0.999992,-1.0,1.0,0.060832,0.161575,-0.615522,1.000000,...,0.742369,1.0,-0.005484,-0.138811,-0.019887,-1.0,0.999411,0.995893,-0.030141,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
140483,d49a66825bb16ceb5b6a01126e1f2391b085dba8da44ee...,0.032123,0.982215,1.000000,-1.0,1.0,0.060832,-0.999824,-0.796037,1.000000,...,-0.990642,1.0,-0.005575,-0.138812,-0.019884,-1.0,0.952706,1.000000,-0.030275,1.0
140484,f772af6720c0b591d49b97946c5e420c1c077affc0f7c7...,0.032044,0.998128,1.000000,-1.0,1.0,0.060832,1.000000,0.928164,-0.999997,...,0.977808,1.0,-0.005480,-0.138808,-0.019887,-1.0,0.870030,1.000000,-0.030142,1.0
140485,06b282335bc4853f888e1ab50a6ba23a8e420d42313959...,0.042038,-0.999993,0.301673,-1.0,1.0,0.060845,0.454959,-0.879582,1.000000,...,0.999729,1.0,-0.005487,-0.138819,-0.018412,-1.0,0.999998,1.000000,-0.029808,1.0
140486,90d423a25d7cdaf674f7d78bc37d88830443ff17717e02...,0.032044,1.000000,1.000000,-1.0,1.0,0.060832,1.000000,1.000000,-0.822283,...,-0.999994,1.0,-0.005480,-0.138813,-0.019887,-1.0,0.844851,1.000000,-0.030147,1.0


In [47]:
not_only_trx.to_parquet("not_only_trx.parquet", index=False, engine="pyarrow", compression="snappy")

# Downstream

Использование эмбеддингов для даунстрим задачи. Для всех таргетов одни и те же параметры бустинга для простоты

In [48]:
class Downstream:
    def __init__(
        self,
        train_path,
        test_path,
        params,
        result_path,
        col_id='client_id',
        targets=(
            'target_1',
            'target_2',
            'target_3',
            'target_4'
        )
    ):
        self.train_path = train_path
        self.test_path = test_path

        self.col_id = col_id
        self.all_targets = targets
        self.params = params
        self.result_path = result_path
        self.drop_feat = list(self.all_targets) + [self.col_id]

    def fit(self):

        train_embeddings = pd.read_parquet(self.train_path)
        X_train = train_embeddings.drop(columns=self.drop_feat)

        clfs = dict()
        for col_target in self.all_targets:
            clf = ltb.LGBMClassifier(**self.params)
            y_train = train_embeddings[col_target]
            clf.fit(X_train, y_train)
            print(f'Model fitted, target: {col_target}')
            clfs[col_target] = clf

        return clfs

    def get_scores(
        self,
        clfs
    ):
        scores = pd.DataFrame([])

        test_embeddings_curr = pd.read_parquet(self.test_path).drop_duplicates('client_id')
        X_test = test_embeddings_curr.drop(columns=[self.col_id])
        ids = test_embeddings_curr[self.col_id]
        scores[self.col_id] = ids

        for col_target in self.all_targets:
            clf = clfs[col_target]
            score = clf.predict_proba(X_test)[:, 1]
            scores[col_target] = score

        return scores

    def run(self):
        clfs = self.fit()
        scores = self.get_scores(clfs)

        scores.to_csv(self.result_path)

        return scores

In [49]:
params = {
    "n_estimators": 500,
      "boosting_type": "gbdt",
      "objective": "binary",
      "subsample": 0.5,
      "subsample_freq": 1,
      "learning_rate": 0.02,
      "feature_fraction": 0.75,
      "max_depth": 6,
      "lambda_l1": 1,
      "lambda_l2": 1,
      "min_data_in_leaf": 50,
      "random_state": 42,
      "n_jobs": 8,
}

dw = Downstream(
    train_path="train.parquet",
    test_path="not_only_trx.parquet",
    params=params,
    result_path='sample_submission.csv'
)

scores = dw.run()
scores

[LightGBM] [Warning] min_data_in_leaf is set=50, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=50
[LightGBM] [Warning] feature_fraction is set=0.75, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.75
[LightGBM] [Warning] lambda_l2 is set=1, reg_lambda=0.0 will be ignored. Current value: lambda_l2=1
[LightGBM] [Warning] lambda_l1 is set=1, reg_alpha=0.0 will be ignored. Current value: lambda_l1=1
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] min_data_in_leaf is set=50, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=50
[LightGBM] [Warning] feature_fraction is set=0.75, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.75
[LightGBM] [Warning] lambda_l2 is set=1, reg_lambda=0.0 will be ignored. Current value: lambda_l2=1
[LightGBM] [Warning] lambda_l1 is set=1, reg_alpha=0.0 will be ignored. Cur

[LightGBM] [Warning] min_data_in_leaf is set=50, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=50
[LightGBM] [Warning] feature_fraction is set=0.75, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.75
[LightGBM] [Warning] lambda_l2 is set=1, reg_lambda=0.0 will be ignored. Current value: lambda_l2=1
[LightGBM] [Warning] lambda_l1 is set=1, reg_alpha=0.0 will be ignored. Current value: lambda_l1=1
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] min_data_in_leaf is set=50, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=50
[LightGBM] [Warning] feature_fraction is set=0.75, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.75
[LightGBM] [Warning] lambda_l2 is set=1, reg_lambda=0.0 will be ignored. Current value: lambda_l2=1
[LightGBM] [Warning] lambda_l1 is set=1, reg_alpha=0.0 will be ignored. Cur

,client_id,target_1,target_2,target_3,target_4
0,2b7ff0c1c99cefe259ed83c5dfa0a403f2cbc88032b671...,0.000650,0.000175,0.071703,0.005033
1,0433d23e224b7a520656da6181efadb8d556bb293158c9...,0.015820,0.000524,0.014175,0.003590
2,f2ce8b292e5f9f778f3e20db7608ac76dc8812113a2631...,0.014127,0.005294,0.019212,0.006896
3,4f807e8b163c653bcaeff9f925983568f4c3e6b1a1f231...,0.009759,0.005368,0.023235,0.034852
4,64369f6f8ae1b719332ee1bfb2b454e642b2053d2c9b8a...,0.008663,0.000584,0.011120,0.002960
...,...,...,...,...,...
140483,d49a66825bb16ceb5b6a01126e1f2391b085dba8da44ee...,0.011789,0.000743,0.018081,0.005627
140484,f772af6720c0b591d49b97946c5e420c1c077affc0f7c7...,0.008849,0.000366,0.006720,0.010273
140485,06b282335bc4853f888e1ab50a6ba23a8e420d42313959...,0.009516,0.002302,0.008874,0.001547
140486,90d423a25d7cdaf674f7d78bc37d88830443ff17717e02...,0.020638,0.000718,0.015884,0.031294
